In [1]:
# Qiskit Simulation - Matrix Product State 

import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit.providers.aer import QasmSimulator

# Construct quantum circuit
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)
circ.measure([0,1], [0,1])

# Select the QasmSimulator from the Aer provider
simulator = QasmSimulator(method='matrix_product_state')

# Execute and get counts, using the matrix_product_state method
result = execute(circ, simulator).result()
counts = result.get_counts(circ)
counts

{'00': 526, '11': 498}

Matrix_product_state in QsamSimulator determines a particular the internal representation of the quantum state. The method is based on the concept of matrix product states. 
Relevant Papers :  https://arxiv.org/abs/quant-ph/0301063, https://arxiv.org/abs/1008.3477.

A pure quantum state is usually described as a state vector, by the expression
|ψ⟩=∑1i1=0…∑1in=0ci1…in|ii⟩⊗…⊗|in⟩

The state vector representation implies an exponential size representation, regardless of the actual circuit. Every quantum gate operating on this representation requires exponential time and memory.

The matrix product state (MPS) representation offers a local representation, in the form: 
Γ[1]λ[1]Γ[2]λ[2]…Γ[1]λ[n−1]Γ[n]
, such that all the information contained in the ci1…in

, can be generated out of the MPS representation.

Every Γ[i]
is a tensor of complex numbers that represents qubit i. Every λ[i] is a matrix of real numbers that is used to normalize the amplitudes of qubits i and i+1

. Single-qubit gates operate only on the relevant tensor.

Two-qubit gates operate on consecutive qubits i
and i+1. This involves a tensor-contract operation over λ[i−1], Γ[i−1], λ[i], Γ[i+1] and λ[i+1]

, that creates a single tensor. We apply the gate to this tensor, and then decompose back to the original structure. This operation may increase the size of the respective tensors. Gates that involve two qubits that are not consecutive, require a series of swap gates to bring the two qubits next to each other and then the reverse swaps.

In the worst case, the tensors may grow exponentially. However, the size of the overall structure remains ‘small’ for circuits that do not have ‘many’ two-qubit gates. This allows much more efficient operations in circuits with relatively ‘low’ entanglement. Characterizing when to use this method over other methods is a subject of current research.
Using the matrix product state simulation method¶

The matrix product state simulation method is invoked in the QasmSimulator by setting the simulation method. Other than that, all operations are controlled by the QasmSimulator itself, as in the following example:

In [2]:
from qiskit.extensions.simulator.snapshot import snapshot

In [5]:
circ = QuantumCircuit(2, 2)
circ.h(0)
circ.cx(0, 1)

# Define a snapshot that shows the current state vector
circ.snapshot('my_sv', snapshot_type='statevector')
circ.measure([0,1], [0,1])

# Execute
job_sim = execute(circ, simulator)
result = job_sim.result()

#print the state vector
result.data()['snapshots']['statevector']['my_sv'][0]

array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])

In [6]:
result.get_counts()

{'00': 502, '11': 522}

Running circuits using the matrix product state simulation method can be fast, relative to other methods. However, if we generate the state vector during the execution, then the conversion to state vector is, of course, exponential in memory and time, and therefore we don’t benefit from using this method. We can benefit if we only do operations that don’t require printing the full state vector. For example, if we run a circuit and then take measurement. The circuit below has 200 qubits. We create an EPR state involving all these qubits. Although this state is highly entangled, it is handled well by the matrix product state method, because there are effectively only two states.

We can handle more qubits than this, but execution may take a few minutes. Try running a similar circuit with 500 qubits! Or maybe even 1000 (you can get a cup of coffee while waiting).

In [8]:
num_qubits = 100
circ = QuantumCircuit(num_qubits, num_qubits)

# Create EPR state
circ.h(0)
for i in range (0, num_qubits-1):
    circ.cx(i, i+1)

# Measure
circ.measure(range(num_qubits), range(num_qubits))

job_sim = execute(circ, simulator)
result = job_sim.result()
print("Time taken: {} sec".format(result.time_taken))
result.get_counts()

Time taken: 1.0404067039489746 sec


{'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000': 522,
 '1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111': 502}